<a href="https://colab.research.google.com/github/RyoNakamae/finance_getdata/blob/main/finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yahoo_finance_api2


  Created wheel for yahoo-finance-api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3731 sha256=bb57f5482f6e8ed98a1844ab35875a8ba04b74b940c276a25a99c4a7ac3de119
  Stored in directory: /root/.cache/pip/wheels/df/83/e1/df62dc96545acda58900be732f6eca5f34c84f28c9bd882f75
Successfully built yahoo-finance-api2


In [ ]:
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as datetime
import pandas as pd

def get_data(code):

  stock = share.Share(code)
  symbol_data = None

  try:
      # get_historical(period_type, period, frequency_type, frequency)
      symbol_data = stock.get_historical(share.PERIOD_TYPE_DAY, 10, share.FREQUENCY_TYPE_DAY, 1)
  except YahooFinanceError as e:
      print(f">>> [{code}] : {e.message}")
      return None

  # 日付処理のためにDataframeに変換
  df_symbol_data = pd.DataFrame(symbol_data)
  # タイムスタンプを変換
  df_symbol_data["datetime"] = pd.to_datetime(df_symbol_data.timestamp, unit="ms")
  # 日本時間へ変換
  df_symbol_data["datetime_jst"] = df_symbol_data["datetime"] + datetime.timedelta(hours=9)
  # 後でキーに使いやすいように日本時間を文字列に変換
  df_symbol_data["datetime_jst_str"] = df_symbol_data["datetime_jst"].apply(lambda x: x.strftime('%Y%m%d'))

  return df_symbol_data

In [ ]:
# 日本株については.Tをつける：code + ".T"
stockList = ["AAPL","AMZN","FB","MSFT","sdasd","GOOGL","4661.T"]

# とりあえずの格納先：'code_日付'をキーとし、終値を格納する
# 取得できなかった場合はSKIPする
dic_stock = {}
for code in stockList:
  res = get_data(code)
  if(res is None):
    print(f">>> [{code}] : NoData : SKIP")
    continue

  print(code)
  # 終値
  num = len(res)
  for index in range(len(res)):
    dic_target = res[index:index+1].to_dict()
    dic_stock[code + "_" + dic_target["datetime_jst_str"][index]] = dic_target["close"][index]

print(dic_stock)

AAPL
AMZN
FB
MSFT
>>> [sdasd] : Not Found: No data found, symbol may be delisted
>>> [sdasd] : NoData : SKIP
GOOGL
4661.T
{'AAPL_20220127': 159.22000122070312, 'AAPL_20220128': 170.3300018310547, 'AAPL_20220131': 174.77999877929688, 'AAPL_20220201': 174.61000061035156, 'AAPL_20220202': 175.83999633789062, 'AAPL_20220203': 172.89999389648438, 'AAPL_20220204': 172.38999938964844, 'AMZN_20220127': 2792.75, 'AMZN_20220128': 2879.56005859375, 'AMZN_20220131': 2991.469970703125, 'AMZN_20220201': 3023.8701171875, 'AMZN_20220202': 3012.25, 'AMZN_20220203': 2776.909912109375, 'AMZN_20220204': 3152.7900390625, 'FB_20220127': 294.6400146484375, 'FB_20220128': 301.7099914550781, 'FB_20220131': 313.260009765625, 'FB_20220201': 319.0, 'FB_20220202': 323.0, 'FB_20220203': 237.75999450683594, 'FB_20220204': 237.08999633789062, 'MSFT_20220127': 299.8399963378906, 'MSFT_20220128': 308.260009765625, 'MSFT_20220131': 310.9800109863281, 'MSFT_20220201': 308.760009765625, 'MSFT_20220202': 313.4599914550781,